## Using SVR and NN to predict 2013 Capicity percentage

In [72]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn import cross_validation
from sklearn import grid_search
from sklearn import preprocessing
from sklearn import svm
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout, Activation
from keras.optimizers import SGD
from keras.preprocessing import text

In [73]:
d= pd.read_excel("reactors-operating.xls",encoding= "ISO-8859-1")

In [74]:
d.head()

,US_Commercial Nuclear Power Reactors - Operating Reactors,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
0,"Plant Name, Unit Number",NRC Reactor Unit Web Page,Docket Number,License Number,Location,NRC Region,Licensee,Parent Company Utility Name,Parent Company Website,Parent Company Notes,...,2013 Capacity Factor (Percent),2012 Capacity Factor (Percent),2011 Capacity Factor (Percent),2010 Capacity Factor (Percent),2009 Capacity Factor (Percent),2008 Capacity Factor (Percent),NaN,2005 Capacity Factor (Percent),2004 Capacity Factor (Percent),2003 Capacity Factor (Percent)
1,"Arkansas Nuclear One, Unit 1",ANO 1,05000313,DPR-51,"London, AR (6 MI WNW of Russellville, AR)",4,"Entergy Operations, Inc.","Entergy Nuclear Operations, Inc.",www.entergy-nuclear.com,NaN,...,0.560015,1.02,0.87,0.9,0.99,0.83,NaN,0.78,0.92,0.92
2,"Arkansas Nuclear One, Unit 2",ANO 2,05000368,NPF-6,"London, AR (6 MI WNW of Russellville, AR)",4,"Entergy Operations, Inc.","Entergy Nuclear Operations, Inc.",www.entergy-nuclear.com,NaN,...,0.909777,0.93,0.9,0.97,0.9,0.911003,NaN,0.91,0.99,0.9
3,"Beaver Valley Power Station, Unit 1",Beaver Valley 1,05000334,DPR-66,"Shippingport, PA(17 MI W of McCandless, PA)",1,First Energy Nuclear Operating Co.,First Energy Nuclear Operating Company,www.firstenergycorp.com,NaN,...,0.860891,0.92,1.01,0.91,0.92,1.01408,NaN,1.01,0.93,0.83
4,"Beaver Valley Power Station, Unit 2",Beaver Valley 2,05000412,NPF-73,"Shippingport, PA (17 MI W of McCandless, PA)",1,First Energy Nuclear Operating Co.,First Energy Nuclear Operating Company,www.firstenergycorp.com,NaN,...,0.966991,0.91,1.02,0.84,0.87,1.03,NaN,0.93,1,0.91


In [75]:
d.columns = d.iloc[0]

In [76]:
d

,"Plant Name, Unit Number",NRC Reactor Unit Web Page,Docket Number,License Number,Location,NRC Region,Licensee,Parent Company Utility Name,Parent Company Website,Parent Company Notes,...,2013 Capacity Factor (Percent),2012 Capacity Factor (Percent),2011 Capacity Factor (Percent),2010 Capacity Factor (Percent),2009 Capacity Factor (Percent),2008 Capacity Factor (Percent),nan,2005 Capacity Factor (Percent),2004 Capacity Factor (Percent),2003 Capacity Factor (Percent)
0,"Plant Name, Unit Number",NRC Reactor Unit Web Page,Docket Number,License Number,Location,NRC Region,Licensee,Parent Company Utility Name,Parent Company Website,Parent Company Notes,...,2013 Capacity Factor (Percent),2012 Capacity Factor (Percent),2011 Capacity Factor (Percent),2010 Capacity Factor (Percent),2009 Capacity Factor (Percent),2008 Capacity Factor (Percent),NaN,2005 Capacity Factor (Percent),2004 Capacity Factor (Percent),2003 Capacity Factor (Percent)
1,"Arkansas Nuclear One, Unit 1",ANO 1,05000313,DPR-51,"London, AR (6 MI WNW of Russellville, AR)",4,"Entergy Operations, Inc.","Entergy Nuclear Operations, Inc.",www.entergy-nuclear.com,NaN,...,0.560015,1.02,0.87,0.9,0.99,0.83,NaN,0.78,0.92,0.92
2,"Arkansas Nuclear One, Unit 2",ANO 2,05000368,NPF-6,"London, AR (6 MI WNW of Russellville, AR)",4,"Entergy Operations, Inc.","Entergy Nuclear Operations, Inc.",www.entergy-nuclear.com,NaN,...,0.909777,0.93,0.9,0.97,0.9,0.911003,NaN,0.91,0.99,0.9
3,"Beaver Valley Power Station, Unit 1",Beaver Valley 1,05000334,DPR-66,"Shippingport, PA(17 MI W of McCandless, PA)",1,First Energy Nuclear Operating Co.,First Energy Nuclear Operating Company,www.firstenergycorp.com,NaN,...,0.860891,0.92,1.01,0.91,0.92,1.01408,NaN,1.01,0.93,0.83
4,"Beaver Valley Power Station, Unit 2",Beaver Valley 2,05000412,NPF-73,"Shippingport, PA (17 MI W of McCandless, PA)",1,First Energy Nuclear Operating Co.,First Energy Nuclear Operating Company,www.firstenergycorp.com,NaN,...,0.966991,0.91,1.02,0.84,0.87,1.03,NaN,0.93,1,0.91
5,"Braidwood Station, Unit 1",Braidwood 1,05000456,NPF-72,"Braceville, IL (20 MI SSW of Joliet, IL)",3,"Exelon Generation Co., LLC","Exelon Corporation, LLC",www.exeloncorp.com,NaN,...,0.95,0.91,1.01,0.89,0.95,1.01399,NaN,1,0.95,0.97
6,"Braidwood Station, Unit 2",Braidwood 2,05000457,NPF-77,"Braceville, IL (20 MI SSW of Joliet, IL)",3,"Exelon Generation Co., LLC","Exelon Corporation, LLC",www.exeloncorp.com,NaN,...,0.98,0.93,0.93,0.99,0.93,0.92143,NaN,0.94,1.01,0.96
7,"Browns Ferry Nuclear Plant, Unit 1",Browns Ferry 1,05000259,DPR-33,"Limestone County, AL (10 miles south of Athens...",2,Tenessee Valley Authority,Tennessee Valley Authority,www.tva.gov,NaN,...,0.939996,0.88,0.91,0.86,0.94,0.877119,NaN,NaN,NaN,NaN
8,"Browns Ferry Nuclear Plant, Unit 2",Browns Ferry 2,05000260,DPR-52,"Limestone County, AL (10 miles south of Athens...",2,Tenessee Valley Authority,Tennessee Valley Authority,www.tva.gov,NaN,...,0.792529,0.99,0.8,0.91,0.94,0.98,NaN,0.9,1,0.86
9,"Browns Ferry Nuclear Plant, Unit 3",Browns Ferry 3,05000296,DPR-68,"Limestone County, AL (10 miles south of Athens...",2,Tenessee Valley Authority,Tennessee Valley Authority,www.tva.gov,NaN,...,0.885835,0.83,0.87,0.81,0.95,0.811608,NaN,0.94,0.89,0.96


In [77]:
d=d.drop(d.index[0])
d

,"Plant Name, Unit Number",NRC Reactor Unit Web Page,Docket Number,License Number,Location,NRC Region,Licensee,Parent Company Utility Name,Parent Company Website,Parent Company Notes,...,2013 Capacity Factor (Percent),2012 Capacity Factor (Percent),2011 Capacity Factor (Percent),2010 Capacity Factor (Percent),2009 Capacity Factor (Percent),2008 Capacity Factor (Percent),nan,2005 Capacity Factor (Percent),2004 Capacity Factor (Percent),2003 Capacity Factor (Percent)
1,"Arkansas Nuclear One, Unit 1",ANO 1,05000313,DPR-51,"London, AR (6 MI WNW of Russellville, AR)",4,"Entergy Operations, Inc.","Entergy Nuclear Operations, Inc.",www.entergy-nuclear.com,NaN,...,0.560015,1.02,0.87,0.9,0.99,0.83,NaN,0.78,0.92,0.92
2,"Arkansas Nuclear One, Unit 2",ANO 2,05000368,NPF-6,"London, AR (6 MI WNW of Russellville, AR)",4,"Entergy Operations, Inc.","Entergy Nuclear Operations, Inc.",www.entergy-nuclear.com,NaN,...,0.909777,0.93,0.9,0.97,0.9,0.911003,NaN,0.91,0.99,0.9
3,"Beaver Valley Power Station, Unit 1",Beaver Valley 1,05000334,DPR-66,"Shippingport, PA(17 MI W of McCandless, PA)",1,First Energy Nuclear Operating Co.,First Energy Nuclear Operating Company,www.firstenergycorp.com,NaN,...,0.860891,0.92,1.01,0.91,0.92,1.01408,NaN,1.01,0.93,0.83
4,"Beaver Valley Power Station, Unit 2",Beaver Valley 2,05000412,NPF-73,"Shippingport, PA (17 MI W of McCandless, PA)",1,First Energy Nuclear Operating Co.,First Energy Nuclear Operating Company,www.firstenergycorp.com,NaN,...,0.966991,0.91,1.02,0.84,0.87,1.03,NaN,0.93,1,0.91
5,"Braidwood Station, Unit 1",Braidwood 1,05000456,NPF-72,"Braceville, IL (20 MI SSW of Joliet, IL)",3,"Exelon Generation Co., LLC","Exelon Corporation, LLC",www.exeloncorp.com,NaN,...,0.95,0.91,1.01,0.89,0.95,1.01399,NaN,1,0.95,0.97
6,"Braidwood Station, Unit 2",Braidwood 2,05000457,NPF-77,"Braceville, IL (20 MI SSW of Joliet, IL)",3,"Exelon Generation Co., LLC","Exelon Corporation, LLC",www.exeloncorp.com,NaN,...,0.98,0.93,0.93,0.99,0.93,0.92143,NaN,0.94,1.01,0.96
7,"Browns Ferry Nuclear Plant, Unit 1",Browns Ferry 1,05000259,DPR-33,"Limestone County, AL (10 miles south of Athens...",2,Tenessee Valley Authority,Tennessee Valley Authority,www.tva.gov,NaN,...,0.939996,0.88,0.91,0.86,0.94,0.877119,NaN,NaN,NaN,NaN
8,"Browns Ferry Nuclear Plant, Unit 2",Browns Ferry 2,05000260,DPR-52,"Limestone County, AL (10 miles south of Athens...",2,Tenessee Valley Authority,Tennessee Valley Authority,www.tva.gov,NaN,...,0.792529,0.99,0.8,0.91,0.94,0.98,NaN,0.9,1,0.86
9,"Browns Ferry Nuclear Plant, Unit 3",Browns Ferry 3,05000296,DPR-68,"Limestone County, AL (10 miles south of Athens...",2,Tenessee Valley Authority,Tennessee Valley Authority,www.tva.gov,NaN,...,0.885835,0.83,0.87,0.81,0.95,0.811608,NaN,0.94,0.89,0.96
10,"Brunswick Steam Electric Plant, Unit 1",Brunswick 1,05000325,DPR-71,"Southport, NC (20 MI S of Wilmington, NC)",2,"Duke Energy Progress, Inc.",Duke Energy,www.duke-energy.com,NaN,...,0.919873,0.77,1,0.83,0.98,0.853949,NaN,0.94,0.86,1.01


In [78]:
d

,"Plant Name, Unit Number",NRC Reactor Unit Web Page,Docket Number,License Number,Location,NRC Region,Licensee,Parent Company Utility Name,Parent Company Website,Parent Company Notes,...,2013 Capacity Factor (Percent),2012 Capacity Factor (Percent),2011 Capacity Factor (Percent),2010 Capacity Factor (Percent),2009 Capacity Factor (Percent),2008 Capacity Factor (Percent),nan,2005 Capacity Factor (Percent),2004 Capacity Factor (Percent),2003 Capacity Factor (Percent)
1,"Arkansas Nuclear One, Unit 1",ANO 1,05000313,DPR-51,"London, AR (6 MI WNW of Russellville, AR)",4,"Entergy Operations, Inc.","Entergy Nuclear Operations, Inc.",www.entergy-nuclear.com,NaN,...,0.560015,1.02,0.87,0.9,0.99,0.83,NaN,0.78,0.92,0.92
2,"Arkansas Nuclear One, Unit 2",ANO 2,05000368,NPF-6,"London, AR (6 MI WNW of Russellville, AR)",4,"Entergy Operations, Inc.","Entergy Nuclear Operations, Inc.",www.entergy-nuclear.com,NaN,...,0.909777,0.93,0.9,0.97,0.9,0.911003,NaN,0.91,0.99,0.9
3,"Beaver Valley Power Station, Unit 1",Beaver Valley 1,05000334,DPR-66,"Shippingport, PA(17 MI W of McCandless, PA)",1,First Energy Nuclear Operating Co.,First Energy Nuclear Operating Company,www.firstenergycorp.com,NaN,...,0.860891,0.92,1.01,0.91,0.92,1.01408,NaN,1.01,0.93,0.83
4,"Beaver Valley Power Station, Unit 2",Beaver Valley 2,05000412,NPF-73,"Shippingport, PA (17 MI W of McCandless, PA)",1,First Energy Nuclear Operating Co.,First Energy Nuclear Operating Company,www.firstenergycorp.com,NaN,...,0.966991,0.91,1.02,0.84,0.87,1.03,NaN,0.93,1,0.91
5,"Braidwood Station, Unit 1",Braidwood 1,05000456,NPF-72,"Braceville, IL (20 MI SSW of Joliet, IL)",3,"Exelon Generation Co., LLC","Exelon Corporation, LLC",www.exeloncorp.com,NaN,...,0.95,0.91,1.01,0.89,0.95,1.01399,NaN,1,0.95,0.97
6,"Braidwood Station, Unit 2",Braidwood 2,05000457,NPF-77,"Braceville, IL (20 MI SSW of Joliet, IL)",3,"Exelon Generation Co., LLC","Exelon Corporation, LLC",www.exeloncorp.com,NaN,...,0.98,0.93,0.93,0.99,0.93,0.92143,NaN,0.94,1.01,0.96
7,"Browns Ferry Nuclear Plant, Unit 1",Browns Ferry 1,05000259,DPR-33,"Limestone County, AL (10 miles south of Athens...",2,Tenessee Valley Authority,Tennessee Valley Authority,www.tva.gov,NaN,...,0.939996,0.88,0.91,0.86,0.94,0.877119,NaN,NaN,NaN,NaN
8,"Browns Ferry Nuclear Plant, Unit 2",Browns Ferry 2,05000260,DPR-52,"Limestone County, AL (10 miles south of Athens...",2,Tenessee Valley Authority,Tennessee Valley Authority,www.tva.gov,NaN,...,0.792529,0.99,0.8,0.91,0.94,0.98,NaN,0.9,1,0.86
9,"Browns Ferry Nuclear Plant, Unit 3",Browns Ferry 3,05000296,DPR-68,"Limestone County, AL (10 miles south of Athens...",2,Tenessee Valley Authority,Tennessee Valley Authority,www.tva.gov,NaN,...,0.885835,0.83,0.87,0.81,0.95,0.811608,NaN,0.94,0.89,0.96
10,"Brunswick Steam Electric Plant, Unit 1",Brunswick 1,05000325,DPR-71,"Southport, NC (20 MI S of Wilmington, NC)",2,"Duke Energy Progress, Inc.",Duke Energy,www.duke-energy.com,NaN,...,0.919873,0.77,1,0.83,0.98,0.853949,NaN,0.94,0.86,1.01


In [79]:
d.columns

Index([                       'Plant Name, Unit Number',
                           'NRC Reactor Unit Web Page ',
                                       'Docket  Number',
                                       'License Number',
                                             'Location',
                                           'NRC Region',
                                             'Licensee',
                          'Parent Company Utility Name',
                               'Parent Company Website',
                                 'Parent Company Notes',
                        'Reactor and Containment Type ',
       'Nuclear Steam System Supplier and Design Type ',
                                   'Architect-Engineer',
                                           'Contructor',
                           'Construction Permit Issued',
                             'Operating License Issued',
                                 'Commercial Operation',
                     'Renewed O

In [80]:
df=d[['2013 Capacity Factor (Percent)',
                       '2012 Capacity Factor (Percent)',
                       '2011 Capacity Factor (Percent)',
                       '2010 Capacity Factor (Percent)',
                       '2009 Capacity Factor (Percent)',
                       '2008 Capacity Factor (Percent)',
                       '2005 Capacity Factor (Percent)',
                       '2004 Capacity Factor (Percent)',
                       '2003 Capacity Factor (Percent)',"Contructor","Architect-Engineer","Reactor and Containment Type ","Nuclear Steam System Supplier and Design Type ","NRC Region"]]

In [81]:
df

,2013 Capacity Factor (Percent),2012 Capacity Factor (Percent),2011 Capacity Factor (Percent),2010 Capacity Factor (Percent),2009 Capacity Factor (Percent),2008 Capacity Factor (Percent),2005 Capacity Factor (Percent),2004 Capacity Factor (Percent),2003 Capacity Factor (Percent),Contructor,Architect-Engineer,Reactor and Containment Type,Nuclear Steam System Supplier and Design Type,NRC Region
1,0.560015,1.02,0.87,0.9,0.99,0.83,0.78,0.92,0.92,BECH,BECH,PWR-DRYAMB,B&W LLP,4
2,0.909777,0.93,0.9,0.97,0.9,0.911003,0.91,0.99,0.9,BECH,BECH,PWR-DRYAMB,CE,4
3,0.860891,0.92,1.01,0.91,0.92,1.01408,1.01,0.93,0.83,S&W,S&W,PWR-DRYAMB,WEST 3LP,1
4,0.966991,0.91,1.02,0.84,0.87,1.03,0.93,1,0.91,S&W,S&W,PWR-DRYAMB,WEST 3LP,1
5,0.95,0.91,1.01,0.89,0.95,1.01399,1,0.95,0.97,CWE,S&L,PWR-DRYAMB,WEST 4LP,3
6,0.98,0.93,0.93,0.99,0.93,0.92143,0.94,1.01,0.96,CWE,S&L,PWR-DRYAMB,WEST 4LP,3
7,0.939996,0.88,0.91,0.86,0.94,0.877119,NaN,NaN,NaN,TVA,TVA,BWR-MARK 1,GE 4,2
8,0.792529,0.99,0.8,0.91,0.94,0.98,0.9,1,0.86,TVA,TVA,BWR-MARK 1,GE 4,2
9,0.885835,0.83,0.87,0.81,0.95,0.811608,0.94,0.89,0.96,TVA,TVA,BWR-MARK 1,GE 4,2
10,0.919873,0.77,1,0.83,0.98,0.853949,0.94,0.86,1.01,BRRT,UE&C,BWR-MARK 1,GE 4,2


df["2013 Capacity Factor (Percent)"]=df['2013 Capacity Factor (Percent)'].str.replace("%","")
df["2012 Capacity Factor (Percent)"]=df['2012 Capacity Factor (Percent)'].str.replace("%","")
df["2011 Capacity Factor (Percent)"]=df['2011 Capacity Factor (Percent)'].str.replace("%","")
df["2010 Capacity Factor (Percent)"]=df['2010 Capacity Factor (Percent)'].str.replace("%","")
df["2009 Capacity Factor (Percent)"]=df['2009 Capacity Factor (Percent)'].str.replace("%","")
df["2008 Capacity Factor (Percent)"]=df['2008 Capacity Factor (Percent)'].str.replace("%","")
df["2005 Capacity Factor (Percent)"]=df['2005 Capacity Factor (Percent)'].str.replace("%","")
df["2004 Capacity Factor (Percent)"]=df['2004 Capacity Factor (Percent)'].str.replace("%","")
df["2003 Capacity Factor (Percent)"]=df['2003 Capacity Factor (Percent)'].str.replace("%","")

In [84]:
df=df.dropna(axis=0)

In [85]:
df

,2013 Capacity Factor (Percent),2012 Capacity Factor (Percent),2011 Capacity Factor (Percent),2010 Capacity Factor (Percent),2009 Capacity Factor (Percent),2008 Capacity Factor (Percent),2005 Capacity Factor (Percent),2004 Capacity Factor (Percent),2003 Capacity Factor (Percent),Contructor,Architect-Engineer,Reactor and Containment Type,Nuclear Steam System Supplier and Design Type,NRC Region
1,0.560015,1.02,0.87,0.9,0.99,0.83,0.78,0.92,0.92,BECH,BECH,PWR-DRYAMB,B&W LLP,4
2,0.909777,0.93,0.9,0.97,0.9,0.911003,0.91,0.99,0.9,BECH,BECH,PWR-DRYAMB,CE,4
3,0.860891,0.92,1.01,0.91,0.92,1.01408,1.01,0.93,0.83,S&W,S&W,PWR-DRYAMB,WEST 3LP,1
4,0.966991,0.91,1.02,0.84,0.87,1.03,0.93,1,0.91,S&W,S&W,PWR-DRYAMB,WEST 3LP,1
5,0.95,0.91,1.01,0.89,0.95,1.01399,1,0.95,0.97,CWE,S&L,PWR-DRYAMB,WEST 4LP,3
6,0.98,0.93,0.93,0.99,0.93,0.92143,0.94,1.01,0.96,CWE,S&L,PWR-DRYAMB,WEST 4LP,3
8,0.792529,0.99,0.8,0.91,0.94,0.98,0.9,1,0.86,TVA,TVA,BWR-MARK 1,GE 4,2
9,0.885835,0.83,0.87,0.81,0.95,0.811608,0.94,0.89,0.96,TVA,TVA,BWR-MARK 1,GE 4,2
10,0.919873,0.77,1,0.83,0.98,0.853949,0.94,0.86,1.01,BRRT,UE&C,BWR-MARK 1,GE 4,2
11,0.727035,0.98,0.79,0.99,0.8,0.953211,0.86,0.98,0.99,BRRT,UE&C,BWR-MARK 1,GE 4,2


In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 1 to 99
Data columns (total 14 columns):
2013 Capacity Factor (Percent)                    96 non-null object
2012 Capacity Factor (Percent)                    96 non-null object
2011 Capacity Factor (Percent)                    96 non-null object
2010 Capacity Factor (Percent)                    96 non-null object
2009 Capacity Factor (Percent)                    96 non-null object
2008 Capacity Factor (Percent)                    96 non-null object
2005 Capacity Factor (Percent)                    96 non-null object
2004 Capacity Factor (Percent)                    96 non-null object
2003 Capacity Factor (Percent)                    96 non-null object
Contructor                                        96 non-null object
Architect-Engineer                                96 non-null object
Reactor and Containment Type                      96 non-null object
Nuclear Steam System Supplier and Design Type     96 non-null object


In [94]:
df["Contructor"].nunique()

18

In [95]:
# create dummy variable for "Contructor"

In [96]:
x10= pd.get_dummies(df["Contructor"]).values

In [97]:
df["Architect-Engineer"].nunique()

18

In [98]:
x11= pd.get_dummies(df["Architect-Engineer"]).values

In [100]:
x12=pd.get_dummies(df["Reactor and Containment Type "]).values

In [103]:
x13=pd.get_dummies(df["Nuclear Steam System Supplier and Design Type "]).values

In [104]:
x14=pd.get_dummies(df["NRC Region"]).values

In [106]:
df= df[['2013 Capacity Factor (Percent)',
                       '2012 Capacity Factor (Percent)',
                       '2011 Capacity Factor (Percent)',
                       '2010 Capacity Factor (Percent)',
                       '2009 Capacity Factor (Percent)',
                       '2008 Capacity Factor (Percent)',
                       '2005 Capacity Factor (Percent)',
                       '2004 Capacity Factor (Percent)',
                       '2003 Capacity Factor (Percent)']]

In [107]:
y=  df['2013 Capacity Factor (Percent)']

In [108]:
df_final= np.concatenate((df,x10),axis=1)

In [109]:
df_final= np.concatenate((df_final,x11),axis=1)

In [110]:
df_final= np.concatenate((df_final,x12),axis=1)

In [111]:
df_final= np.concatenate((df_final,x13),axis=1)

In [112]:
df_final= np.concatenate((df_final,x14),axis=1)

In [113]:
df_final

array([[0.5600151954165972, 1.02, 0.87, ..., 0, 0, 1],
       [0.9097774033130445, 0.93, 0.9, ..., 0, 0, 1],
       [0.8608905099017004, 0.92, 1.01, ..., 0, 0, 0],
       ..., 
       [0.8911483066971081, 0.77, 0.82, ..., 0, 0, 1],
       [0.8959661848775753, 0.87, 0.84, ..., 1, 0, 0],
       [0.64534637295997, 0.8, 0.72, ..., 0, 0, 1]], dtype=object)

In [114]:
df_final=df_final.astype(np.float64)

In [115]:
y=y.astype(np.float64)/100

In [116]:
min_max_scaler= preprocessing.MinMaxScaler()

In [117]:
df_final= min_max_scaler.fit_transform(df_final)

In [118]:
clf= svm.SVR()

In [119]:
clf.fit(df_final,y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [120]:
svr= clf.predict(df_final)[:,np.newaxis]

In [121]:
svr

array([[ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0.00778866],
       [ 0

In [122]:
q=np.mean(np.abs(clf.predict(df_final)-y))

In [123]:
q

0.0011798472049429508

In [124]:
print("Mean abs SVR error"+str(q))

Mean abs SVR error0.0011798472049429508
